In [35]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from aurora import Aurora
from aurora.normalisation import locations, scales
import xarray as xr
from aurora import  Batch, Metadata, rollout
import torch
from scipy.ndimage import zoom

In [36]:
# File paths
co2_data_sfc = r"C:\Users\DC\Documents\vscode4\data_sfc.nc"
download_path = r'C:\Users\DC\Documents\vscode4\ubuntu_vs\vscode\VScode2\vscode\down'
co2_data_sfc2 = r"C:\Users\DC\Documents\vscode4\7bc2ad758dd66a384b66b64346e658f3\data_sfc.nc"

# Load datasets
co2_data_sfc = xr.open_dataset(co2_data_sfc, engine="netcdf4")
static_vars_ds = xr.open_dataset(f"{download_path}/static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(f"{download_path}/2023-01-01-surface-level.nc", engine="netcdf4")
atmos_vars_ds = xr.open_dataset(f"{download_path}/2023-01-01-atmospheric.nc", engine="netcdf4")
co2_data_sfc2 = xr.open_dataset(co2_data_sfc2, engine="netcdf4")

In [37]:
tuple(int(level) for level in atmos_vars_ds.pressure_level.values)

(1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50)

In [38]:
co2_data_sfc

<xarray.Dataset>
Dimensions:                  (forecast_period: 41, forecast_reference_time: 1,
                              latitude: 1801, longitude: 3600)
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 00:00:00 ... 5...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2024-11-01
  * latitude                 (latitude) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * longitude                (longitude) float64 0.0 0.1 0.2 ... 359.8 359.9
Data variables:
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] ...
    aco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-06T01:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [39]:
static_vars_ds

<xarray.Dataset>
Dimensions:     (valid_time: 1, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 ...
  * valid_time  (valid_time) datetime64[ns] 2023-01-01
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      object ...
Data variables:
    z           (valid_time, latitude, longitude) float32 ...
    lsm         (valid_time, latitude, longitude) float32 ...
    slt         (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-07T09:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [40]:
surf_vars_ds

<xarray.Dataset>
Dimensions:     (valid_time: 4, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 ...
  * valid_time  (valid_time) datetime64[ns] 2023-01-01 ... 2023-01-01T18:00:00
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) object ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 ...
    u10         (valid_time, latitude, longitude) float32 ...
    v10         (valid_time, latitude, longitude) float32 ...
    msl         (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-07T09:02 GRIB to CDM+CF via cfgrib-0.9.1...

In [41]:
surf_vars_ds["t2m"].values.shape

(4, 721, 1440)

In [42]:
surf_vars_ds["t2m"].values[[0,1]].shape

(2, 721, 1440)

In [43]:
surf_vars_ds["t2m"].values[[0,1]][None].shape

(1, 2, 721, 1440)

In [44]:
co2_data_sfc['aco2rec'][:,0,:,:].values.shape

(41, 1801, 3600)

In [ ]:
arr1=co2_data_sfc['aco2rec'][0,0,:,:].values.shape

(1801, 3600)

In [74]:
arr1[None].shape

(1, 1801, 3600)

In [70]:
co2_data_sfc['aco2rec'].values.shape

(41, 1, 1801, 3600)

In [71]:
co2_data_sfc2['aco2rec'].values.shape

(41, 22, 1801, 3600)

In [45]:
co2_data_sfc['aco2rec'][:,0,:,:][[0,1]].values.shape

(2, 1801, 3600)

In [46]:
arr1=co2_data_sfc['aco2rec'][:,0,:,:][[0,1]].values

In [47]:
arr1=arr1[None]

In [48]:
arr1.shape

(1, 2, 1801, 3600)

In [49]:
atmos_vars_ds["t"].values[[0,1]].shape

(2, 13, 721, 1440)

In [50]:
atmos_vars_ds["t"].shape

(4, 13, 721, 1440)

In [51]:
atmos_vars_ds["t"].values[[0,1]][None].shape

(1, 2, 13, 721, 1440)

In [52]:
static_vars_ds["z"].values[0].shape

(721, 1440)

In [53]:
len(surf_vars_ds.valid_time.values.astype("datetime64[s]").tolist())

4

In [54]:
len(co2_data_sfc.valid_time.values.astype("datetime64[s]").tolist())

1

In [55]:
arr_aco2rec=co2_data_sfc['aco2rec'][:,0,:,:][[0,1]].values
print(arr_aco2rec[None].shape)
arr_aco2gpp=co2_data_sfc['aco2gpp'][:,0,:,:][[0,1]].values
print(arr_aco2gpp[None].shape)
arr_aco2nee=co2_data_sfc['aco2nee'][:,0,:,:][[0,1]].values
print(arr_aco2nee[None].shape)
arr_fco2rec=co2_data_sfc['fco2rec'][:,0,:,:][[0,1]].values
print(arr_fco2rec[None].shape)
arr_fco2gpp=co2_data_sfc['fco2gpp'][:,0,:,:][[0,1]].values
print(arr_fco2gpp[None].shape)
arr_fco2nee=co2_data_sfc['fco2nee'][:,0,:,:][[0,1]].values
print(arr_fco2nee[None].shape)

(1, 2, 1801, 3600)
(1, 2, 1801, 3600)
(1, 2, 1801, 3600)
(1, 2, 1801, 3600)
(1, 2, 1801, 3600)
(1, 2, 1801, 3600)


In [56]:
static_vars_ds["z"].values[0]
arr1=zoom(static_vars_ds["z"].values[0], (1801 / static_vars_ds["z"].shape[1], 3600 / static_vars_ds["z"].shape[2]), order=1)

In [57]:
type(arr1)

numpy.ndarray

In [58]:
atmos_vars_ds["t"].values[[0, 1]][None].shape

(1, 2, 13, 721, 1440)

In [59]:
atmos_vars_ds["t"].values[[0, 1]][None]
arr2=zoom(atmos_vars_ds["t"].values[[0, 1]][None], (1, 1,1, 1801 / atmos_vars_ds["t"].shape[2], 3600 / atmos_vars_ds["t"].shape[3]), order=1)

In [60]:
arr2.shape

(1, 2, 13, 1801, 3600)

In [61]:
surf_vars_ds.latitude.values.shape

(721,)

In [62]:
co2_data_sfc.latitude.values.shape

(1801,)

In [63]:
co2_data_sfc['aco2rec'][:,0,:,:][[0,1]].shape

(2, 1801, 3600)

In [64]:
import numpy as np
from scipy.ndimage import zoom

co2_data_sfc['aco2rec'][:,0,:,:][[0,1]]
# Upsample the array using bilinear interpolation
upsampled_array = zoom(co2_data_sfc['aco2rec'][:,0,:,:][[0,1]], (1,  721 / co2_data_sfc['aco2rec'].shape[2], 1440 / co2_data_sfc['aco2rec'].shape[3]), order=1)

In [65]:
upsampled_array.shape

(2, 721, 1440)

In [66]:
surf_vars_ds.valid_time.values.astype("datetime64[s]").tolist()

[datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2023, 1, 1, 6, 0),
 datetime.datetime(2023, 1, 1, 12, 0),
 datetime.datetime(2023, 1, 1, 18, 0)]

In [67]:
co2_data_sfc.valid_time.values.astype("datetime64[s]").tolist()[0]

[datetime.datetime(2024, 11, 1, 0, 0),
 datetime.datetime(2024, 11, 1, 3, 0),
 datetime.datetime(2024, 11, 1, 6, 0),
 datetime.datetime(2024, 11, 1, 9, 0),
 datetime.datetime(2024, 11, 1, 12, 0),
 datetime.datetime(2024, 11, 1, 15, 0),
 datetime.datetime(2024, 11, 1, 18, 0),
 datetime.datetime(2024, 11, 1, 21, 0),
 datetime.datetime(2024, 11, 2, 0, 0),
 datetime.datetime(2024, 11, 2, 3, 0),
 datetime.datetime(2024, 11, 2, 6, 0),
 datetime.datetime(2024, 11, 2, 9, 0),
 datetime.datetime(2024, 11, 2, 12, 0),
 datetime.datetime(2024, 11, 2, 15, 0),
 datetime.datetime(2024, 11, 2, 18, 0),
 datetime.datetime(2024, 11, 2, 21, 0),
 datetime.datetime(2024, 11, 3, 0, 0),
 datetime.datetime(2024, 11, 3, 3, 0),
 datetime.datetime(2024, 11, 3, 6, 0),
 datetime.datetime(2024, 11, 3, 9, 0),
 datetime.datetime(2024, 11, 3, 12, 0),
 datetime.datetime(2024, 11, 3, 15, 0),
 datetime.datetime(2024, 11, 3, 18, 0),
 datetime.datetime(2024, 11, 3, 21, 0),
 datetime.datetime(2024, 11, 4, 0, 0),
 datetime.dat

In [68]:
co2_data_sfc2

<xarray.Dataset>
Dimensions:                  (forecast_period: 41, forecast_reference_time: 22,
                              latitude: 1801, longitude: 3600)
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 00:00:00 ... 5...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2024-10...
  * latitude                 (latitude) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * longitude                (longitude) float64 0.0 0.1 0.2 ... 359.8 359.9
Data variables:
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] ...
    aco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-06T03:46 GRIB to CDM+CF via cfgrib-0.9.1...

In [69]:
co2_data_sfc


<xarray.Dataset>
Dimensions:                  (forecast_period: 41, forecast_reference_time: 1,
                              latitude: 1801, longitude: 3600)
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 00:00:00 ... 5...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2024-11-01
  * latitude                 (latitude) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * longitude                (longitude) float64 0.0 0.1 0.2 ... 359.8 359.9
Data variables:
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] ...
    aco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    aco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2rec                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2gpp                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
    fco2nee                  (forecast_period, forecast_reference_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-06T01:33 GRIB to CDM+CF via cfgrib-0.9.1...